In [8]:
import requests
from bs4 import BeautifulSoup

In [10]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive'
}

In [2]:
def get_review_data_from_html(html):
  stars_label = "small stars stars-"
  stars_start = html.index(stars_label) + len(stars_label)
  if (html[stars_start + 1] == '"'):
    stars = float(html[stars_start])
  else:
    stars = float(html[stars_start] + "." + html[stars_start+1])

  review_span_idx = html.index('<span id="texto-review-')
  parsed_html = html[review_span_idx:]
  review_start = parsed_html.index(">")
  review_end = parsed_html.index("</span>")
  review = parsed_html[review_start + 1: review_end]

  return stars, review

In [3]:
import json
def save_list_to_json(obj, path):
  # Convert list to JSON string
  json_data = json.dumps(obj, indent=4)

  # Save JSON to a file
  with open(path, "w") as json_file:
      json_file.write(json_data)

def load_list_from_json(path):
  with open(path, "r") as json_file:
      return json.load(json_file)

In [18]:
book_links_local_path = "/content/book_links.json"
labeled_data_local_path = "/content/labeled_data.json"

In [19]:
load_from_file = True

if load_from_file:
  labeled_data =load_list_from_json(labeled_data_local_path)
else:
  labeled_data = []

book_links = set(load_list_from_json(book_links_local_path))

In [20]:
import math

total_book_links = len(book_links)
batch_size = 50
num_batches = math.ceil(total_book_links / batch_size)

In [ ]:
labeled_data = []

for j in range(0, num_batches):
    start = j * batch_size

    for i, book_link in enumerate(sorted(list(book_links))[start:start + batch_size]):
        if (i % 25 == 0): print(start + i)

        try:
            response = requests.get(book_link, headers=headers, timeout=5)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Grab title (assuming it's still inside the static HTML)
            title_elem = soup.find(class_="tituloProducto")
            if not title_elem:
                print("No title found")
                continue
            title = title_elem.get_text(strip=True)

            # Reviews (limited to reviews already on the page)
            review_substring = "review-n-"
            review_elements = soup.find_all(class_=lambda x: x and review_substring in x)

            for review_element in review_elements:
                review_html = review_element.decode_contents()
                stars, review = get_review_data_from_html(review_html)
                labeled_data.append(
                    {
                        "rating": stars,
                        "review": review,
                        "title": title,
                    }
                )
        except Exception as e:
            print(f"Error processing {book_link}: {e}")
            continue
    save_list_to_json(labeled_data, labeled_data_local_path)


0
25
50


In [ ]:
save_list_to_json(labeled_data, labeled_data_local_path)

In [ ]:
len (labeled_data)

12140

In [21]:
num_batches

40

We good for the ROBOTS.TXT-- https://www.buscalibre.cl/robots.txt